In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.streaming import StreamingQuery
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import *

: 

In [ ]:
spark = (
    SparkSession.builder.master("local[1]")
    .appName("Tutorial App")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1")
    .getOrCreate()
)

: 

In [ ]:
smart_thermo= spark.readStream \
                          .format("kafka") \
                          .option("kafka.bootstrap.servers", "kafka:29092") \
                          .option("subscribe", "smartthermo")

: 

In [ ]:
smart_thermo=smart_thermo.load()

: 

In [ ]:
smart_thermo_schema = StructType([
  StructField("room_id", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("temperature", FloatType())
  ])

: 

In [ ]:
smart_thermo= smart_thermo.withColumn("message_content", F.from_json(F.col("value").cast("string"),smart_thermo_schema))

: 

In [ ]:
smart_thermo = smart_thermo.selectExpr("CAST(value AS STRING)")
smart_thermo = smart_thermo.select(from_json(smart_thermo.value, smart_thermo_schema).alias("data"))
smart_thermo = smart_thermo.select("data.*")

: 

In [ ]:
query = smart_thermo.writeStream \
  .outputMode("append") \
  .format("console") \
  .start()

: 

In [ ]:
query

: 

: 